## 1. Presentación del problema

### Breve resumen del contexto
Cada año el Draft de la NBA selecciona a un número limitado de jugadores de entre cientos de prospectos del basquetbol colegial estadounidense. La **Atlantic Coast Conference (ACC)** es una de las conferencias más importantes de la División I de la NCAA e incluye programas de alto perfil como Duke, North Carolina, Virginia, Clemson, entre otros, que históricamente aportan muchos jugadores al Draft.

Sin embargo, no todos los jugadores destacados de la ACC terminan siendo elegidos por un equipo NBA. Contar con un modelo que, a partir de sus características físicas, estadísticas y antecedentes de reclutamiento, estime la probabilidad de que un jugador de la ACC sea drafteado puede ayudar a visores, entrenadores y analistas a priorizar talento y tomar mejores decisiones.

### Justificación: ¿por qué este problema es adecuado para aplicar vecinos cercanos?
El problema se formula como una **clasificación binaria**:

- Clase 1: jugador **drafteado** en el Draft 2025.
- Clase 0: jugador **no drafteado**.

Para cada jugador de la ACC se tienen múltiples **variables numéricas** (puntos, rebotes, asistencias por partido, peso, ranking RSCI, etc.) y **categóricas** (posición, clase académica, universidad). Es razonable suponer que:

- Jugadores con perfiles y estadísticas similares tienden a tener resultados parecidos respecto al draft (ser o no seleccionados).

El algoritmo de **K Vecinos Más Cercanos (KNN)** clasifica precisamente a partir de la similitud entre instancias, medida mediante una **distancia** en el espacio de características. Además:

- No requiere suponer una forma específica de la distribución de los datos.
- Es intuitivo de explicar: “este jugador se parece a estos k jugadores, de los cuales la mayoría sí/no fue drafteada”.

Por estas razones, el problema de predecir si un jugador de la ACC será drafteado es adecuado para la aplicación de KNN.

### Objetivo del análisis
> Construir un modelo de clasificación basado en K Vecinos Más Cercanos que, a partir de características físicas, estadísticas y de reclutamiento de los jugadores de la Atlantic Coast Conference (ACC) en la temporada 2024-25, prediga si un jugador será seleccionado en el Draft 2025 de la NBA.


---

## 2. Recolección de datos

### Fuente de datos

1. **Historial del Draft 2025**
   - Se utilizó una tabla pública con los **60 picks del Draft 2025** que incluye: número de selección (`Pk`), nombre del jugador (`Player`) y universidad (`College`).
   - De esta tabla se filtraron únicamente los jugadores cuyo `College` pertenece a la ACC.

2. **Rosters y estadísticas de la ACC 2024-25**
   - Se obtuvieron las plantillas y estadísticas de todos los equipos de la ACC para la temporada 2024-25 desde páginas de estadísticas de baloncesto universitario (por ejemplo, Sports-Reference College Basketball).
   - Para cada jugador se descargaron columnas como: `Player`, `Class`, `Pos`, `Height`, `Weight`, `RSCI Top 100`, `Summary`, entre otras.

### Formato de los datos
- Las tablas se exportaron/copiarion desde las páginas web y se guardaron como archivos **CSV**.
- Posteriormente se integraron en un único archivo llamado, `acc_2024_25_draft2025_knn.csv`, utilizando **Python (pandas)** para unir y transformar los datos.

### Variables relevantes y descripción

Cada fila del dataset final representa **un jugador de la ACC en 2024-25**. Las principales variables son:

- `Player`: nombre del jugador (identificador, no se usa como predictor).
- `School`: universidad a la que pertenece (equipos de la ACC).
- `Class`: año académico (`FR`, `SO`, `JR`, `SR`).
- `Pos`: posición en la cancha (`G`, `F`, `C`, etc.).
- `Height`: estatura en formato pies-pulgadas (ej. `6-9`).
- `Weight`: peso en libras.
- `RSCI Top 100`: ranking de reclutamiento en preparatoria (ej. `1 (2024)`), si está disponible.
- `Summary`: texto con un resumen estadístico por partido (ej. `19.2 Pts 7.5 Reb 4.2 Ast`).
- `PTS_pg`, `REB_pg`, `AST_pg`: puntos, rebotes y asistencias por partido extraídos de `Summary`.
- `RSCI_rank_filled`: ranking RSCI transformado a número (1, 2, …; se usa un valor alto como 200 para los no rankeados).
- `is_top100`: variable binaria (1 si aparece en el Top 100, 0 si no).
- `drafted`: **variable objetivo**, binaria (1 si el jugador aparece en el Draft 2025 con una universidad de la ACC, 0 en caso contrario).

---


In [2]:
import pandas as pd

# 1. Cargar archivos
stats = pd.read_csv("stats.csv")          # stats ACC
draft = pd.read_csv("draft_2025.csv")     # 60 picks


## 3. Preparación de los datos

### Limpieza de datos

- Se revisaron y limpiaron los nombres de columnas para eliminar espacios y mantener un formato consistente.
- Para cada equipo de la ACC se añadió la columna `School` y luego se concatenaron todos los rosters en un solo DataFrame.
- Se eliminaron columnas irrelevantes para el modelo, como `Hometown`, `High School` y otros campos textuales de contexto que no se usarían como predictores.
- Se verificó que no hubiera **filas duplicadas** por jugador y universidad.
- Los valores faltantes en variables clave se trataron de la siguiente manera:
  - `RSCI_rank`: se extrajo el número del texto `RSCI Top 100` y, cuando no había ranking, se asignó un valor alto (200) para indicar “no rankeado”.
  - `is_top100`: se definió como 1 si había información en `RSCI Top 100` y 0 si estaba vacío.
  - Otras variables numéricas se revisaron y, cuando fue necesario, se imputaron o se dejaron como `NaN` para luego excluir columnas poco completas.

### Transformaciones

- Se construyó una llave `KEY` concatenando `Player` y `School` en mayúsculas, tanto en la tabla de la ACC como en la del Draft 2025.
- Se realizó un **join** (left join en pandas) entre el DataFrame de jugadores de la ACC y la tabla filtrada del Draft (solo universidades ACC).
  - Si la llave `KEY` aparecía en la tabla de Draft, se copiaba su `Pk` y se marcaba `drafted = 1`.
  - Si no aparecía, se asignaba `drafted = 0`.
- La columna `Summary` se procesó con expresiones regulares para extraer:
  - `PTS_pg` → primer número antes de “Pts”.
  - `REB_pg` → segundo número antes de “Reb”.
  - `AST_pg` → tercer número antes de “Ast”.



---

In [3]:
# 1. Limpiar espacios en nombres de columnas
stats.columns = stats.columns.str.strip()
draft.columns = draft.columns.str.strip()

In [4]:
# 2. Obtener la lista de escuelas de la ACC que realmente aparecen en stats
acc_schools = stats['School'].dropna().str.strip().unique()
print("Escuelas ACC en stats:", acc_schools)

Escuelas ACC en stats: ['Duke' 'Clemson' 'SMU' 'Wake Forest' 'UNC' 'Stanford' 'Georgia Tech'
 'Florida State' 'Pitt' 'Virginia' 'Notre Dame' 'Virginia Tech' 'Syracuse'
 'California' 'NC State' 'Boston College' 'Miami (FL)']


In [5]:
# 2.1 Filtrar el draft solo a jugadores de esas escuelas
draft_acc = draft[draft['College'].isin(acc_schools)].copy()

# 2.2. Renombrar 'Pk' a 'Pick' para que el nombre sea más claro
draft_acc = draft_acc.rename(columns={'Pk': 'Pick'})

# 3. Crear la KEY en ambos dataframes
def make_key(player_series, school_series):
    return (player_series.str.upper().str.strip()
            + "|" +
            school_series.str.upper().str.strip())

# Asegúrate de que en stats tienes columna 'School' con 'Duke' o 'Clemson'
stats['KEY'] = make_key(stats['Player'], stats['School'])
draft_acc['KEY'] = make_key(draft_acc['Player'], draft_acc['College'])

# 4. Hacer el join (left join desde stats hacia draft_dc)
merged = stats.merge(
    draft_acc[['KEY', 'Pick']],  # solo necesitamos KEY y pick
    on='KEY',
    how='left'
)

# 5. Crear columna drafted (1 si tiene pick, 0 si no)
merged['drafted'] = merged['Pick'].notna().astype(int)

In [6]:
# 6. Extraer PTS, REB, AST desde la columna 'Summary'
# Ejemplo de formato esperado: "19.2 Pts 7.5 Reb 4.2 Ast"
pattern = r'(?P<PTS>\d+\.?\d*)\s+Pts\s+(?P<REB>\d+\.?\d*)\s+Reb\s+(?P<AST>\d+\.?\d*)\s+Ast'

summary_stats = merged['Summary'].str.extract(pattern)

# 6.1 Convertir a numérico y renombrar bonito
merged['PTS_pg'] = pd.to_numeric(summary_stats['PTS'], errors='coerce')
merged['REB_pg'] = pd.to_numeric(summary_stats['REB'], errors='coerce')
merged['AST_pg'] = pd.to_numeric(summary_stats['AST'], errors='coerce')


In [7]:
# Eliminamos la columna Summary original
merged = merged.drop(columns=['Summary'])

In [8]:
# Extraer solo el número antes del paréntesis, por ejemplo 1 de "1 (2024)"
merged['RSCI_rank'] = (
    merged['RSCI Top 100']
    .astype(str)                      # por si hay NaN
    .str.extract(r'(\d+)')[0]         # toma solo la primera coincidencia numérica
)

# Convertir a numérico (NaN si no se puede convertir)
merged['RSCI_rank'] = pd.to_numeric(merged['RSCI_rank'], errors='coerce')


In [9]:
# Para que el modelo sepa si el jugador siquiera aparecía en el Top 100:
merged['is_top100'] = merged['RSCI_rank'].notna().astype(int)

In [10]:
# Para evitar NaN en RSCI_rank, le darmos un valor “alto” a los que no están rankeados, por ejemplo 200
merged['RSCI_rank_filled'] = merged['RSCI_rank'].fillna(200)

In [11]:
# Dataset final listo para KNN
# 7. Elegimos las columnas que sí usaremos en el modelo
cols_modelo = [
    'Player', 'School', 'Class', 'Pos', 'Height', 'Weight',
    'PTS_pg', 'REB_pg', 'AST_pg', 'RSCI_rank_filled',
     'is_top100',
    'drafted'
]
dataset_knn = merged[cols_modelo].copy()

# 8. Guardamos CSV final
dataset_knn.to_csv("dataset_nba_duke_clemson_knn.csv", index=False)


### Normalización y discretización

- Antes de aplicar KNN en Altair AI Studio se utilizó un operador de **Normalize** para escalar las variables numéricas (`Weight`, `PTS_pg`, `REB_pg`, `AST_pg`, `RSCI_rank_filled`, etc.), de forma que todas quedaran en una escala comparable y la distancia euclidiana no estuviera dominada por una sola variable.
- Se exploró la discretización de algunas variables (por ejemplo, rangos de peso o de puntos por partido) solo con fines descriptivos. Sin embargo, para el modelo KNN se trabajó con las versiones continuas normalizadas, ya que esto conserva mejor la información para el cálculo de distancias.

## 4. Aplicación de reglas de distancia

### Descripción breve del algoritmo utilizado

El algoritmo **K-Vecinos Más Cercanos (KNN)** clasifica una instancia nueva buscando sus **k vecinos más cercanos** dentro del conjunto de entrenamiento, de acuerdo con una medida de distancia, usualmente la **distancia euclidiana** en el espacio de características.

En este proyecto:

1. Cada jugador de la ACC se representa como un vector con sus características numéricas normalizadas:
   `Weight`, `PTS_pg`, `REB_pg`, `AST_pg`, `RSCI_rank_filled`, etc.
2. Para cada jugador del conjunto de prueba, el modelo calcula la distancia a todos los jugadores del conjunto de entrenamiento y selecciona los **k** más cercanos.
3. La clase que predomina entre esos k vecinos se asigna al jugador evaluado.

### Ejecución en Altair AI Studio

El experimento se implementó en Altair AI Studio mediante el siguiente flujo:

1. **Retrieve**: carga del dataset `acc_2024_25_draft2025_knn.csv`.
2. **Normalize**: normalización de las variables numéricas seleccionadas.
3. **Split Data**: división del conjunto de datos en entrenamiento y prueba (70% para entrenamiento y 30% para prueba, con muestreo aleatorio).
4. **K-NN**: entrenamiento del modelo de K-Vecinos Cercanos como **clasificador**, usando:
   - `drafted_label` como atributo de etiqueta.
   - Variables numéricas normalizadas como atributos de entrada.
   - Un valor de k definido experimentalmente (k = 2), que se puede ajustar y comparar.
5. **Apply Model**: aplicación del modelo entrenado al subconjunto de prueba.
6. **Performance**: cálculo de métricas de desempeño del clasificador, incluyendo **accuracy**, **precisión por clase**, **recall** y la **matriz de confusión**.


---


## 5. Análisis de resultados

Al evaluar el modelo en el conjunto de prueba, Altair AI Studio generó una **matriz de confusión** y métricas de desempeño globales (accuracy, precisión y recall por clase).

En términos generales, el modelo KNN mostró una buena capacidad para distinguir entre jugadores de la ACC que fueron seleccionados en el Draft 2025 y los que no, especialmente cuando se incluyeron como variables:

- Las estadísticas por partido (`PTS_pg`, `REB_pg`, `AST_pg`).
- El peso y, en algunos casos, la estatura (convertida a pulgadas).
- Los indicadores de reclutamiento (`RSCI_rank_filled`, `is_top100`).

Se observó que:

- Los jugadores drafteados tienden a concentrarse en regiones del espacio de características con **mayor producción ofensiva** (más puntos y rebotes) y mejores indicadores de reclutamiento (menor ranking RSCI, es decir, prospectos más valorados en preparatoria).
- La normalización ayudó a que ninguna variable dominara la distancia, permitiendo que KNN tomara en cuenta tanto el rendimiento como el perfil físico y de reclutamiento.

El valor de **k** se puede ajustar (por ejemplo, probando k = 3, 5, 7) para equilibrar entre un modelo muy sensible al ruido (k pequeño) y uno demasiado “suavizado” (k grande). En las pruebas realizadas, valores intermedios de k ofrecieron un desempeño estable y una matriz de confusión con pocos errores en ambas clases.

En el reporte se documentan las métricas concretas obtenidas (accuracy y distribución de aciertos/errores en la matriz de confusión) para la partición train/test empleada.

---

## 6. Conclusiones

### Patrones o conocimientos descubiertos

- Dentro de la ACC, los jugadores que llegan al Draft 2025 tienden a presentar **mejores estadísticas por partido**, especialmente en puntos y rebotes, en comparación con sus compañeros que no son seleccionados.
- El ranking de reclutamiento (`RSCI_rank_filled`) y la pertenencia al Top 100 (`is_top100`) también resultan informativos: los jugadores que fueron prospectos muy bien rankeados en preparatoria tienen mayor probabilidad de ser drafteados, incluso cuando sus estadísticas universitarias no son extremadamente altas.
- El enfoque basado en similitud (KNN) permite identificar “perfiles de jugador” que históricamente han sido atractivos para equipos NBA: combinaciones de producción ofensiva, tamaño físico y reputación previa.

### ¿Cómo podrían aprovecharse estos resultados?

- Los cuerpos técnicos universitarios podrían usar un modelo de este tipo como **herramienta de apoyo** para identificar qué jugadores de la ACC tienen un perfil más cercano al de ex-jugadores que fueron drafteados, ayudando a:
  - Priorizar su desarrollo individual.
  - Darles mayor visibilidad en partidos clave o torneos.
  - Argumentar con datos objetivos el potencial de ciertos jugadores frente a scouts y reclutadores.
- Las oficinas de scouting podrían complementar su análisis cualitativo con una **vista cuantitativa** basada en similitud con jugadores previamente exitosos.

### Limitaciones y posibles mejoras

- **Cobertura limitada**: aunque se incluyó toda la ACC, solo se consideró **una temporada (2024-25)** y un solo draft (2025). Esto hace que los resultados estén condicionados a esa generación específica de jugadores.
- **Desbalance de clases**: el número de jugadores no drafteados es mucho mayor que el de drafteados, lo que puede sesgar al modelo hacia la clase negativa. Se podría mejorar aplicando técnicas de balanceo (submuestreo, sobremuestreo o pesos de clase).
- **Variables disponibles**: no se incorporaron estadísticas avanzadas (eficiencia ofensiva/defensiva, PER, rating defensivo, etc.) ni información cualitativa (lesiones, rol en el equipo, impresiones de scouts), que también influyen en la decisión de los equipos NBA.
- **Generalización**: el modelo fue entrenado solo con datos de la ACC; su capacidad para generalizar a otras conferencias o a jugadores internacionales es limitada.

Como trabajo futuro se propone:

- Ampliar el dataset a **múltiples temporadas** y **varias conferencias** (SEC, Big Ten, etc.) para obtener más casos positivos y negativos.
- Comparar el desempeño de KNN con otros algoritmos de clasificación (regresión logística, árboles de decisión, random forest, redes neuronales sencillas) usando las mismas variables.
- Explorar la incorporación de estadísticas avanzadas y métricas defensivas, así como descripciones cualitativas, para capturar mejor la complejidad real del proceso de selección en el Draft de la NBA.